In [1]:
import random
import json
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input
from tensorflow.keras.optimizers import SGD

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
lemmatizer=WordNetLemmatizer()
intents=json.loads(open('intents.json').read())
words=[]
classes=[]
documents=[]
ignore_letters=['?','!',',','.']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))

In [8]:
training=[]
output_empty=[0]*len(classes)
for document in documents:
    bag=[]
    word_patterns=document[0]
    word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])
random.shuffle(training)
training=np.array(training, dtype=object)
train_x=list(training[:,0])
train_y=list(training[:,1])

In [9]:
model=Sequential()
model.add(Input(shape=(len(train_x[0]),)))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
model.fit(np.array(train_x),np.array(train_y),epochs=50,batch_size=5, verbose=1)

Epoch 1/50
6/6 [==============================] - 2s 5ms/step - loss: 1.5919 - accuracy: 0.2857
Epoch 2/50
6/6 [==============================] - 0s 4ms/step - loss: 1.6110 - accuracy: 0.2500
Epoch 3/50
6/6 [==============================] - 0s 3ms/step - loss: 1.4789 - accuracy: 0.3214
Epoch 4/50
6/6 [==============================] - 0s 4ms/step - loss: 1.4354 - accuracy: 0.5000
Epoch 5/50
6/6 [==============================] - 0s 5ms/step - loss: 1.3807 - accuracy: 0.5714
Epoch 6/50
6/6 [==============================] - 0s 3ms/step - loss: 1.3071 - accuracy: 0.7143
Epoch 7/50
6/6 [==============================] - 0s 4ms/step - loss: 1.2664 - accuracy: 0.5357
Epoch 8/50
6/6 [==============================] - 0s 3ms/step - loss: 1.1453 - accuracy: 0.6071
Epoch 9/50
6/6 [==============================] - 0s 4ms/step - loss: 1.0551 - accuracy: 0.7143
Epoch 10/50
6/6 [==============================] - 0s 4ms/step - loss: 1.0820 - accuracy: 0.6429
Epoch 11/50
6/6 [======================

In [10]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words=clean_up_sentence(sentence)
    bag=[0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word==w:
                bag[i]=1
    return np.array(bag)

def predict_class(sentence):
    bow=bag_of_words(sentence)
    res=model.predict(np.array([bow]))[0]
    error_threshold=0.25
    results=[[i,r] for i,r in enumerate(res) if r>error_threshold]
    results.sort(key=lambda x:x[1], reverse=True)
    return_list=[]
    for r in results:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list

def get_response(intents,intents_json):
    tag=intents[0]['intent']
    list_of_intents=intents_json['intents']
    for i in list_of_intents:
        if i['tag']==tag:
            result=random.choice(i['responses'])
            break
    return result

In [11]:
while True:
    message=input("")
    ints=predict_class(message)
    res=get_response(ints,intents)
    print(res)

hello
1/1 [==============================] - 0s 125ms/step
Good to see you again!
how are you doing
1/1 [==============================] - 0s 23ms/step
Hello!
programming
1/1 [==============================] - 0s 21ms/step
Start with the book named Let us C by Yashwant Kanetkar
why'
1/1 [==============================] - 0s 38ms/step
Hello!
exit
1/1 [==============================] - 0s 19ms/step
Hello!
thanks
1/1 [==============================] - 0s 23ms/step
Hello!
bye
1/1 [==============================] - 0s 21ms/step
Goodbye!

1/1 [==============================] - 0s 19ms/step
Hi there, how can I help?
no help
1/1 [==============================] - 0s 24ms/step
Hi there, how can I help?
no
1/1 [==============================] - 0s 23ms/step
Hi there, how can I help?
bye
1/1 [==============================] - 0s 44ms/step
Sad to see you go :(
thank
1/1 [==============================] - 0s 23ms/step
Hi there, how can I help?
thanks
1/1 [==============================] - 0s 26ms/s

KeyboardInterrupt: Interrupted by user